In [2]:
import pandas as pd
import numpy as np
import nltk
import spacy
nltk.download('stopwords')
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from nltk.stem.lancaster import LancasterStemmer
from nltk import FreqDist
tqdm.pandas(desc="my bar!")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/matheusmbr/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
try:
    nlp = spacy.load('pt_core_news_sm')
except:
    import os
    os.system('python -m spacy download pt_core_news_sm')
    nlp = spacy.load('pt_core_news_sm')

In [4]:
year = 2013
df = pd.read_csv(f'./Raw/{year}.csv', sep=';', encoding ='latin')
df_processed = df[['AN_BASE', 'NM_SUBTIPO_PRODUCAO', 'NM_PROJETO', 'DS_RESUMO', 'DS_PALAVRA_CHAVE', 'DS_ABSTRACT', 'DS_KEYWORD', 'NM_AREA_AVALIACAO']].copy()
del year

In [5]:
df_processed.head()

,AN_BASE,NM_SUBTIPO_PRODUCAO,NM_PROJETO,DS_RESUMO,DS_PALAVRA_CHAVE,DS_ABSTRACT,DS_KEYWORD,NM_AREA_AVALIACAO
0,2013,DISSERTAÇÃO,NaN,A POSSIBILIDADE DA OCORRÊNCIA DE SISMOS NO BRA...,"ANÁLISE SÍSMICA, CONCRETO ARMADO, DETALHAMENTO...",NÃO INFORMADO,NÃO INFORMADO,ENGENHARIAS I
1,2013,DISSERTAÇÃO,NaN,AS OBRAS DE PONTES ESTAIADAS ENVOLVEM UMA TECN...,PONTE ESTAIADA. PONTE DO SABER. ANÁLISE ESTRUT...,NÃO INFORMADO,NÃO INFORMADO,ENGENHARIAS I
2,2013,TESE,HIPERPLASIA DE CÉLULAS NEUROENDRÓCRINAS EM LAC...,A HIPERPLASIA DE CÉLULAS NEUROENDÓCRINAS DO LA...,1. DOENÇAS PULMONARES INTERSTICIAIS  RADIOGRA...,NEUROENDOCRINE CELL HYPERPLASIA OF INFANCY (NE...,NÃO INFORMADO,MEDICINA II
3,2013,TESE,AVALIAÇÃO ATRAVÉS DAS TÉCNICAS AVANÇADAS DE RM...,ESTUDOS RECENTES TEM DEMONSTRADO ALTERAÇÕES DA...,1. NEURALGIA DO TRIGÊMEO - FISIOPATOLOGIA. 2. ...,RECENT STUDIES HAVE DETECTED LOCAL CHANGES OF ...,NÃO INFORMADO,MEDICINA II
4,2013,DISSERTAÇÃO,NaN,DENTRE AS TÉCNICAS MAIS UTILIZADAS NA DETERMIN...,"PLATAFORMAS FIXAS, ESTRUTURAS OFFSHORE, ESTRUT...",NÃO INFORMADO,NÃO INFORMADO,ENGENHARIAS I


In [6]:
df_processed.columns = df_processed.columns.str.slice(3).str.lower()
df_processed.rename(columns={
    'base': 'ano'
}, inplace=True)

for item in list(df_processed.iloc[:, 1:].columns):
    df_processed[f'{item}'] = df_processed[f'{item}'].str.lower()

In [7]:
df_processed.isna().sum()

ano                     0
subtipo_producao        0
projeto             22280
resumo                  0
palavra_chave           0
abstract                7
keyword                 0
area_avaliacao          0
dtype: int64

In [8]:
df_processed.dropna(subset=['abstract'], inplace=True)

df_processed.fillna({ 'projeto': 'não informado' }, inplace=True)

# df_processed.drop(index=df_processed[
#     df_processed.resumo.str.contains("não informado") | df_processed.palavra_chave.str.contains("não informado") | 
#     df_processed.abstract.str.contains("não informado") | df_processed.keyword.str.contains("não informado") | 
#     df_processed.grande_area_conhecimento.str.contains("não informado")
# ].index, inplace=True)
# df_processed.reset_index(drop=True, inplace=True)

In [9]:
idx = df_processed[(df_processed.palavra_chave == df_processed.keyword) | (df_processed.resumo == df_processed.abstract)].index
df_processed.drop(index=idx, inplace=True)
del idx

In [10]:
for item in ['resumo', 'abstract']:
    df_processed[f'{item}'].replace(to_replace ='[0-9]+', value = '', regex = True, inplace=True)
    df_processed[f'{item}'].replace(to_replace ='[,.:;!?]+', value = '', regex = True, inplace=True)
    df_processed[f'{item}'].replace(to_replace ='[/<>()|\+\-\$%&#@\'\"]+', value = '', regex = True, inplace=True)
    df_processed[f'{item}'].replace(to_replace ='\W+', value = ' ', regex = True, inplace=True)

In [11]:
for item in ['palavra_chave', 'keyword']:
    df_processed[f'{item}'].replace(to_replace ='[,.:;!?]+', value = '', regex = True, inplace=True)
    df_processed[f'{item}'].replace(to_replace ='[/<>()|\+\-\$%&#@\'\"]+', value = '', regex = True, inplace=True)
    df_processed[f'{item}'].replace(to_replace ='\W+', value = ' ', regex = True, inplace=True)
    df_processed[f'{item}'].replace(to_replace ='([0-9][. ])', value = ',', regex = True, inplace=True)
    df_processed[f'{item}'].replace(to_replace ='([.])+', value = ',', regex = True, inplace=True)
    df_processed[f'{item}'].replace(to_replace ='(  )+', value = ' ', regex = True, inplace=True)
    df_processed[f'{item}'] = df_processed[f'{item}'].str.lstrip(', ')

In [12]:
df_processed.head()

,ano,subtipo_producao,projeto,resumo,palavra_chave,abstract,keyword,area_avaliacao
0,2013,dissertação,não informado,a possibilidade da ocorrência de sismos no bra...,análise sísmica concreto armado detalhamento s...,não informado,não informado,engenharias i
1,2013,dissertação,não informado,as obras de pontes estaiadas envolvem uma tecn...,ponte estaiada ponte do saber análise estrutur...,não informado,não informado,engenharias i
2,2013,tese,hiperplasia de células neuroendrócrinas em lac...,a hiperplasia de células neuroendócrinas do la...,"doenças pulmonares intersticiais radiografia ,...",neuroendocrine cell hyperplasia of infancy neh...,não informado,medicina ii
3,2013,tese,avaliação através das técnicas avançadas de rm...,estudos recentes tem demonstrado alterações da...,"neuralgia do trigêmeo fisiopatologia ,neuralgi...",recent studies have detected local changes of ...,não informado,medicina ii
4,2013,dissertação,não informado,dentre as técnicas mais utilizadas na determin...,plataformas fixas estruturas offshore estrutur...,não informado,não informado,engenharias i


## Testes

In [13]:
df_test = df_processed.resumo.copy().head(1000)

In [14]:
def remove_stopwords(text: str) -> str:
    text = wordpunct_tokenize(text)
    stop_words = set(stopwords.words('portuguese'))
    new_text = [item for item in text if item not in stop_words]
    return " ".join(set(new_text))

In [15]:
df_test = df_test.progress_apply(remove_stopwords)

my bar!: 100%|██████████| 1000/1000 [00:00<00:00, 6447.43it/s]


In [16]:
def lc_stem(text: str) -> str:
    stemmer = LancasterStemmer()
    words = [stemmer.stem(w) for w in wordpunct_tokenize(text)]
    return " ".join(set(words))

In [17]:
def spc_lem(text: str) -> str:
    words = [token.lemma_ for token in nlp(text) if token.pos_ != 'PUNCT']
    return " ".join(set(words))

In [18]:
lem = df_test.progress_apply(lc_stem)

my bar!: 100%|██████████| 1000/1000 [00:00<00:00, 1386.80it/s]


In [19]:
stem = df_test.progress_apply(spc_lem)

my bar!: 100%|██████████| 1000/1000 [00:15<00:00, 64.83it/s]


In [32]:
def freq_dist(array: list) -> dict:
    text = ' '.join(array)
    return FreqDist(wordpunct_tokenize(text))

In [33]:
frequencia = freq_dist(stem)

frequencia.most_common(100)

[('de', 868),
 ('este', 700),
 ('estudo', 611),
 ('trabalho', 555),
 ('objetivo', 526),
 ('apresentar', 516),
 ('em', 480),
 ('resultado', 457),
 ('esse', 450),
 ('poder', 424),
 ('grande', 413),
 ('realizar', 392),
 ('ser', 382),
 ('sobre', 381),
 ('utilizar', 377),
 ('pesquisa', 354),
 ('relação', 325),
 ('através', 313),
 ('presente', 312),
 ('dois', 312),
 ('processo', 308),
 ('forma', 305),
 ('analisar', 292),
 ('outro', 289),
 ('partir', 288),
 ('avaliar', 282),
 ('análise', 277),
 ('principal', 262),
 ('além', 262),
 ('meio', 248),
 ('desenvolvimento', 237),
 ('sistema', 237),
 ('assim', 224),
 ('todo', 223),
 ('obter', 221),
 ('dar', 219),
 ('ter', 218),
 ('modelo', 216),
 ('método', 213),
 ('ainda', 207),
 ('ano', 204),
 ('bem', 202),
 ('fazer', 202),
 ('novo', 202),
 ('algum', 201),
 ('identificar', 200),
 ('mostrar', 199),
 ('cada', 198),
 ('bom', 198),
 ('base', 192),
 ('considerar', 190),
 ('desenvolver', 188),
 ('área', 187),
 ('importante', 185),
 ('três', 184),
 ('possí